In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
#Lendo datasets
df_train = pd.read_csv('../data/raw/train.csv') 
df_test = pd.read_csv('../data/raw/test.csv')

In [ ]:
#Verificação
df_train.head()

# Verificação Inicial

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
df_train.dtypes

# Tratando inicial dos dados

In [ ]:
#Convertendo coluna de datas e setando index
df_train['date'] = pd.to_datetime(df_train['date'])
df_train = df_train.set_index('date')

# Análise Inicial dos Dados

## Comportamento Geral das vendas

In [ ]:
#Comportamento das vendas: Anual, trimestral, mensal e diario.
vendas_anuais = df_train['sales'].resample('YE').sum()
vendas_trimestrais = df_train['sales'].resample('QE').sum()
vendas_mensais = df_train['sales'].resample('ME').sum() 
vendas_diarias = df_train['sales'].resample('D').sum() 

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=vendas_diarias.index,
    y=vendas_diarias,
    name='Diário'
))

fig.update_layout(
    title_text="Dashboard de Vendas: Anual, Trimestral, Mensal e Diário",
    xaxis_title="Data",
    yaxis_title="Total de Vendas",
    updatemenus=[
        dict(
            type="buttons",
            direction="down",
            active=0,
            x=1.05, 
            y=1.1,
            buttons=list([
                dict(label="Diário",
                     method="update",
                     args=[{"y": [vendas_diarias], "x": [vendas_diarias.index]},
                           {"title": "Visão Diária das Vendas"}]),
                dict(label="Mensal",
                     method="update",
                     args=[{"y": [vendas_mensais], "x": [vendas_mensais.index]},
                           {"title": "Visão Mensal das Vendas"}]),
                dict(label="Trimestral",
                     method="update",
                     args=[{"y": [vendas_trimestrais], "x": [vendas_trimestrais.index]},
                           {"title": "Visão Trimestral das Vendas"}]),
                dict(label="Anual",
                     method="update",
                     args=[{"y": [vendas_anuais], "x": [vendas_anuais.index]},
                           {"title": "Visão Anual das Vendas"}]),
            ]),
        )
    ])

fig.show()

## Tendência, Sazonalidades e Resíduos

In [ ]:
decomposicao_multiplicativa = seasonal_decompose(vendas_mensais, model='multiplicative')

# Plotando os resultados
fig = decomposicao_multiplicativa.plot()
fig.set_size_inches((16, 10)) # Ajustando o tamanho para melhor visualização
fig.suptitle('Decomposição Multiplicativa da Série de Vendas Mensais', y=1.02, fontsize=16)
plt.show()

# Prophet

In [ ]:
from prophet import Prophet
import holidays
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Comportamento

In [ ]:
# Separando dataframe para avaliação do prophet
df_prophet = vendas_diarias.reset_index()
df_prophet.columns = ['ds', 'y']
df_prophet

In [ ]:
# Criação da coluna de feriados
df_feriados = pd.DataFrame([{'holiday': data[1], 'ds': data[0]} for data in holidays.Brazil(years=range(2013, 2018)).items()])
df_feriados['ds'] = pd.to_datetime(df_feriados['ds'])

In [ ]:
#Separação da base em treino e validação
df_train_prophet = df_prophet[df_prophet['ds'].dt.year < 2017]
df_valid_prophet = df_prophet[df_prophet['ds'].dt.year == 2017]

In [ ]:
# Treinamento
modelo_prophet = Prophet(holidays=df_feriados,
                         daily_seasonality=True,
                         yearly_seasonality=True,
                         weekly_seasonality=True)

modelo_prophet.fit(df_prophet)

periodo_futuro = 90
df_futuro = modelo_prophet.make_future_dataframe(periods=periodo_futuro)
forecast = modelo_prophet.predict(df_futuro)

In [ ]:
#Salvando modelo prophet treinado
import joblib
joblib.dump(modelo_prophet, 'prophet.joblib')

In [ ]:
# Plotando o resultado final de forma interativa com Plotly
fig = go.Figure()

# Adicionar a linha de Vendas Reais (Histórico)
fig.add_trace(go.Scatter(
    x=df_prophet['ds'],
    y=df_prophet['y'],
    mode='lines',
    name='Vendas Reais (Histórico)',
    line=dict(color='blue')
))

# Adicionar a linha de Previsões do Prophet
previsao_futura = forecast[forecast['ds'] > df_prophet['ds'].max()]
fig.add_trace(go.Scatter(
    x=previsao_futura['ds'],
    y=previsao_futura['yhat'], # 'yhat' é a previsão
    mode='lines',
    name='Previsões Prophet (2018)',
    line=dict(color='red', dash='dash')
))

# Customizar o layout
fig.update_layout(
    title='Histórico de Vendas Totais vs. Previsões com Prophet',
    xaxis_title='Data',
    yaxis_title='Total de Vendas'
)

fig.show()